In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
import pandas as pd
from backtesting import Backtest
import yfinance as yf
import numpy as np
import pandas_ta as ta
pd.options.mode.chained_assignment = None
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

C:\Users\adamj\anaconda3\envs\vectorbt_python\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
def EMA(values, n):
    return pd.Series(values).ewm(span=n, adjust=False).mean()

In [3]:
class MACD(Strategy):
    ne = 200
    def init(self):
        # Precompute indicators
        self.ema = self.I(EMA, self.data.Close, self.ne)
        #self.daily_rsi = self.I(ta.rsi, pd.Series(self.data.Close), self.n1)
        self.macd = self.I(ta.macd, pd.Series(self.data.Close), 12, 26)
    
    def next(self):
        if (crossover(self.macd[0], self.macd[2]) and self.data.Close > self.ema and
            self.macd[0] and self.macd[2]<0):
            self.buy()
        elif crossover(self.macd[2], self.macd[0]): 
            self.position.close() 
            
        if (crossover(self.macd[2], self.macd[0]) and self.data.Close < self.ema and
            self.macd[0] and self.macd[2]>0):
            self.sell()
        elif crossover(self.macd[0], self.macd[2]): 
            self.position.close() 
    

In [7]:
#yahoo finance data
start_date = '2023-01-01'
end_date = '2023-09-19'
symbol = '^GSPC'
# Download the stock price data from Yahoo Finance API
data = yf.download(symbol, start=start_date, end=end_date,interval='1h')

[*********************100%%**********************]  1 of 1 completed


In [8]:

bt = Backtest(data, MACD, cash=10000,commission=.002)
stats = bt.run()
stats

Start                     2023-01-03 09:30...
End                       2023-09-18 15:30...
Duration                    258 days 05:00:00
Exposure Time [%]                     21.5781
Equity Final [$]                  9559.761828
Equity Peak [$]                  10180.846078
Return [%]                          -4.402382
Buy & Hold Return [%]               16.488896
Return (Ann.) [%]                   -6.175053
Volatility (Ann.) [%]                4.542589
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -6.380646
Avg. Drawdown [%]                   -3.311994
Max. Drawdown Duration      238 days 02:00:00
Avg. Drawdown Duration      119 days 02:00:00
# Trades                                   21
Win Rate [%]                        28.571429
Best Trade [%]                       1.930808
Worst Trade [%]                      -1.22264
Avg. Trade [%]                    

In [9]:
bt.plot()

C:\Users\adamj\anaconda3\envs\vectorbt_python\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\adamj\anaconda3\envs\vectorbt_python\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\adamj\AppData\Local\Temp\ipykernel_2972\651457420.py:1: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()
C:\Users\adamj\anaconda3\envs\vectorbt_python\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\adamj\anaconda3\envs\vectorbt_python\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multi

GridPlot(id='p1770', ...)

In [7]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2,194,220,4394.210821,4679.419922,570.418201,0.064906,2021-10-11,2021-11-16,36 days
1,2,274,285,4544.480977,4384.569824,-319.822305,-0.035188,2022-02-03,2022-02-18,15 days
2,-2,412,443,4124.823664,3771.969971,705.707387,0.085544,2022-08-23,2022-10-06,44 days
3,-2,486,508,3925.413469,3888.570068,73.686802,0.009386,2022-12-07,2023-01-10,34 days
4,2,557,581,4010.044119,4087.780029,155.471820,0.019385,2023-03-22,2023-04-26,35 days
5,2,668,680,4509.340523,4445.129883,-128.421281,-0.014239,2023-08-30,2023-09-18,19 days
